In [1]:
!pip install youtube-transcript-api

In [2]:
from youtube_transcript_api import YouTubeTranscriptApi

transcript = YouTubeTranscriptApi.get_transcript("nSj67MQFnuY")

In [3]:
transcript[:5]

[{'text': 'Canada is at it again the Trudeau',
  'start': 4.92,
  'duration': 4.04},
 {'text': 'government is provoking India again it',
  'start': 6.48,
  'duration': 5.279},
 {'text': 'has put Indian diplomats in Vancouver',
  'start': 8.96,
  'duration': 4.799},
 {'text': 'under surveillance the methods are',
  'start': 11.759,
  'duration': 4.241},
 {'text': 'invasive the Canadians are engaging in',
  'start': 13.759,
  'duration': 4.001}]

In [4]:
!pip install datasets

In [5]:
 from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
 from datasets import load_dataset
 import torch

 processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")
 model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h")
 ds = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")
 input_values = processor(ds[0]["audio"]["array"], return_tensors="pt", padding="longest").input_values
 logits = model(input_values).logits
 predicted_ids = torch.argmax(logits, dim=-1)
 transcription = processor.batch_decode(predicted_ids)


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [6]:
transcription

['BECAUSE YOU ARE SLEEPING INSTEAD OF CONQUERING THE LOVELY ROSE PRINCESS HAS BECOME A FIDDLE WITHOUT A BEAU ILE POOR SHAGGY SITS THERE A COOING DOVE']

In [1]:
from pydub import AudioSegment
import numpy as np
audio = AudioSegment.from_file("/content/vantage-with-palki-sharma.mp3")
audio = audio.set_frame_rate(16000).set_channels(1)
audio_array = np.array(audio.get_array_of_samples(), dtype=np.float32) / (2**15)
input_values = processor(audio_array, return_tensors="pt", sampling_rate=16000, padding="longest").input_values
logits = model(input_values).logits
predicted_ids = torch.argmax(logits, dim=-1)
transcript_pred = processor.batch_decode(predicted_ids)

NameError: name 'processor' is not defined

In [8]:
!pip install pydub

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,

)

dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
sample = dataset[0]["audio"]

result = pipe(sample)
print(result["text"])


TypeError: WhisperForConditionalGeneration.__init__() got an unexpected keyword argument 'return_timestamp'

In [ ]:
import torch
from transformers import AutoProcessor, WhisperForConditionalGeneration
model_id = "openai/whisper-large-v3-turbo"
processor = AutoProcessor.from_pretrained(model_id)
model = WhisperForConditionalGeneration.from_pretrained(model_id)

ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")

inputs = processor(ds[0]["audio"]["array"], return_tensors="pt")
input_features = inputs.input_features

generated_ids = model.generate(inputs=input_features,return_timestamps=True)

transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
transcription